<pre>
Copyright 2021-2023 Boris Shminke

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
</pre>

In [1]:
# first, we should have a file with a TPTP problem
import sys
import os

if sys.version_info.major == 3 and sys.version_info.minor >= 9:
    from importlib.resources import files
else:
    from importlib_resources import files

test_problem = files("gym_saturation").joinpath(
    os.path.join("resources", "TPTP-mock", "Problems", "SET", "SET001-1.p")
)

In [2]:
!cat {test_problem}

%--------------------------------------------------------------------------
% This file is a part of TPTP. For more information see https://www.tptp.org/

% Copyright
% ---------
% The TPTP is copyrighted 1993-onwards,  by Geoff Sutcliffe &  Christian Suttner.
% Verbatim redistribution of the TPTP and parts of the TPTP is permitted provided
% that the redistribution is clearly attributed to the TPTP.  Distribution of any
% modified version or modified part of the TPTP requires permission.

% File     : SET001-1 : TPTP v7.5.0. Released v1.0.0.
% Domain   : Set Theory
% Problem  : Set members are superset members
% Version  : [LS74] axioms.
% English  : A member of a set is also a member of that set's supersets.

% Refs     : [LS74]  Lawrence & Starkey (1974), Experimental Tests of Resol
%          : [WM76]  Wilson & Minker (1976), Resolution, Refinements, and S
% Source   : [SPRFN]
% Names    : ls100 [LS74]
%          : ls100 [WM76]

% Status   : Unsatisfiable
% Rating   : 0.00 v2.0.0
%

In [3]:
# we have a vampire installed in this container
!vampire --version

Vampire 4.7 (commit 2d02e4655 on 2022-07-11 21:15:24 +0200)
Linked with Z3 4.8.13.0 f03d756e086f81f2596157241e0decfb1c982299 z3-4.8.4-5390-gf03d756e0


In [4]:
# then we create and reset a OpenAI Gym environment
import gym

prover_name = "Vampire"
env = gym.make(f"{prover_name}-v0", problem_list=[test_problem])
observation = env.reset()
# we can render the environment in a human readable form
print("starting hypotheses:")
print(env.render(mode="human"))

starting hypotheses:
cnf(1, lemma, member(X0,X2) | ~subset(X1,X2) | ~member(X0,X1), inference(input, [], [])).
cnf(2, lemma, member(member_of_1_not_of_2(X1,X2),X1) | subset(X1,X2), inference(input, [], [])).
cnf(3, lemma, subset(X1,X2) | ~member(member_of_1_not_of_2(X1,X2),X2), inference(input, [], [])).
cnf(4, lemma, subset(X1,X2) | ~equal_sets(X1,X2), inference(input, [], [])).
cnf(5, lemma, subset(X1,X2) | ~equal_sets(X2,X1), inference(input, [], [])).
cnf(6, lemma, equal_sets(X4,X3) | ~subset(X4,X3) | ~subset(X3,X4), inference(input, [], [])).
cnf(7, lemma, equal_sets(b,bb), inference(input, [], [])).
cnf(8, lemma, member(element_of_b,b), inference(input, [], [])).
cnf(9, lemma, ~member(element_of_b,bb), inference(input, [], [])).


In [5]:
def age_agent_episode(env: gym.Env) -> float:
    """
    Interact with a given environment using the 'age' agent.

    'Age' agent always selects clauses for inference
    in the order they appeared in current proof attempt

    :param env: an environment
    :returns: number of steps in the episode and its gain
    """
    action = 0
    done = False
    num_steps = 0
    gain = 0
    observation = env.reset()
    while not done:
        if observation["action_mask"][action] == 1.0:
            observation, reward, done, info = env.step(action)
            gain += reward
            num_steps += 1
        action += 1
    return gain, num_steps

In [6]:
gain, num_steps = age_agent_episode(env)
if gain > 0:
    print(f"Proof found in {num_steps} steps")

Proof found in 16 steps


In [7]:
# the package also provides a utility function
# for extracting only clauses which became parts of the proof
# (some steps might be unnecessary to find the proof)
from gym_saturation.utils import get_tstp_proof

print(get_tstp_proof(env.state["real_obs"]))

cnf(4, lemma, subset(X1,X2) | ~equal_sets(X1,X2), inference(input, [], [])).
cnf(7, lemma, equal_sets(b,bb), inference(input, [], [])).
cnf(1, lemma, member(X0,X2) | ~subset(X1,X2) | ~member(X0,X1), inference(input, [], [])).
cnf(13, lemma, subset(b,bb), inference(resolution, [], [7, 4])).
cnf(21, lemma, member(X0,bb) | ~member(X0,b), inference(resolution, [], [13, 1])).
cnf(8, lemma, member(element_of_b,b), inference(input, [], [])).
cnf(28, lemma, member(element_of_b,bb), inference(resolution, [], [21, 8])).
cnf(9, lemma, ~member(element_of_b,bb), inference(input, [], [])).
cnf(29, lemma, $false, inference(subsumption_resolution, [], [28, 9])).


In [8]:
# we have an iProver in this container
# (built from https://gitlab.com/korovin/iprover/-/commit/32c76fc0ec665a06324d11dc9d4bf902871d5f15)
!iproveropt --help | head


%---------------- iProver v3.7 (post CASC-J11 2022) ----------------%

iproveropt [options] [filename]



%---------------- Input Options ----------------%

  --out_options <none|control|all>


In [9]:
prover_name = "iProver"

In [10]:
import nest_asyncio

# iProver runs asynchronously with the main Python thread
# so we need to enable nested async loops (one is where Jupyter is running)
nest_asyncio.apply()

In [11]:
# and we can do the same experiment as we did with Vampire
env = gym.make(f"{prover_name}-v0", problem_list=[test_problem])
gain, num_steps = age_agent_episode(env)
# the number of steps is different, because this version of iProver
# relies on superposition instead of resolution
if gain > 0:
    print(f"Proof found in {num_steps} steps")

Proof found in 13 steps
